<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/3_GPT_evaluate_bin_finetunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

## Validación del modelo GPT 3.5 con finetning binaro

**Natalia Dedandi**

model="gpt-3.5-turbo-0125"





La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction



In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [2]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [3]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

## Recupero el modelo con finetuning binario

model="gpt-3.5-turbo-0125"

Names model:
ft:gpt-3.5-turbo-0125:personal:hate-speech-v2:9sJSTs9F



In [9]:
model_ft_bin=client.fine_tuning.jobs.retrieve('ftjob-RI7JuIvpL27w3XMLAIQWTNaw')

In [10]:
model_ft_bin

FineTuningJob(id='ftjob-RI7JuIvpL27w3XMLAIQWTNaw', created_at=1722722606, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:hate-speech-v2:9sJSTs9F', finished_at=1722731536, hyperparameters=Hyperparameters(n_epochs=3, batch_size=72, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=['file-LrrG9erjq0phdN2C09yLtKCc'], seed=112634201, status='succeeded', trained_tokens=8146536, training_file='file-Mt9HMlQtIwmiVA8R0OF4aINw', validation_file='file-bBd2qAB8YmmZUUrnkXsVO9wF', estimated_finish=None, integrations=[], user_provided_suffix='hate_speech_v2')

In [12]:

completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal:hate-speech-v2:9sJSTs9F",
  messages=[
    {'role': 'system', 'content': "You are trained to analyze and detect the sentiment of the given text. Only hateful or not hateful is allowed. If the text is empty or is not clear answer not hateful"},
    {"role": "user", "content": "Hay que matarlos a todos"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='hateful', role='assistant', function_call=None, tool_calls=None)


In [24]:
#result_files=['file-LrrG9erjq0phdN2C09yLtKCc']

content = client.files.content('file-LrrG9erjq0phdN2C09yLtKCc')

In [29]:
import base64
base64.b64decode(content.text.encode("utf-8"))

with open("result.csv", "wb") as f:
  f.write(base64.b64decode(content.text.encode("utf-8")))

In [ ]:
metrics=compare_model_predictions(df_hateful['HATEFUL'],df_hateful['sentiment_bin'])

metrix
{'Accuracy': 0.7756325487084546, 'Precision': 0.7820684033352095, 'Recall': 0.7756325487084546, 'f1': 0.7787565014472516}
